In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "2호모델드랍down"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'2호모델드랍down_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
list_data["2012-01"].columns

Index(['BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
       '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
divide_data = tools.dict_data_divide(list_data.copy(), divide_what=['BW금액', 'CB금액','유상증자금액',
'최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', 
'타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
'당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'], divide_into="총자본", return_origianl=True)       
divide_data["2021-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '특수관계인에대한자금대여금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', 'BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트',
       '업종변경카운트', '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액',
       '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
divide_data1 = tools.dict_data_drop(divide_data.copy(), drop_col=['BW금액', 'CB금액','유상증자금액',
'최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
'타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
'당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'], dropna=False)

divide_data1["2012-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '특수관계인에대한자금대여금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본',
       '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [12]:
drop_col_top_y = ["당기순이익/총자본","영업이익/총자본","총자본","영업현금흐름/총자본"]
drop_col_top_q = ["당기순이익/총자본","영업이익/총자본","총자본","매출총이익/총자본"]

drop_col_down_y = ["상호변경카운트","경영권분쟁소송","업종변경카운트","특수관계인에대한자금대여금액/총자본"]

divide_data2 = tools.dict_data_drop(divide_data1.copy(), drop_col=drop_col_down_y, dropna=False)

In [13]:
drop_togehter = tools.dict_data_drop(divide_data2.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '당기순이익/총자본',
       '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본', '판매 및 관리비/총자본',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '차입금의존도', '총자본', '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [14]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:08<00:00, 11.87it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 16,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=16)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 2,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=2)},
 '2014-01': {'num_of_cluseer': 3,
  'random_state': 42,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=42)},
 '2015-01': {'num_of_cluseer': 3,
  'random_state': 6,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=6)},
 '2016-01': {'num_of_cluseer': 3,
  'random_state': 95,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=95)},
 '2017-01': {'num_of_cluseer': 3,
  'random_state': 87,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=87)},
 '2018-01': {'num_of_cluseer': 3,
  'random_state': 99,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=99)},
 '2019-01': {'num_of_cluseer': 3,
  'random_state': 83,
  'model': KMeans(max_iter=500,

#### 모델 기록

In [15]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

906.0
426.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,426.0,484.0,527.0,449.0,618.0,467.0,737.0,735.0,539.0,648.0,482.0
1,549.0,522.0,621.0,586.0,631.0,710.0,517.0,510.0,754.0,787.0,906.0
2,619.0,599.0,472.0,624.0,482.0,610.0,611.0,674.0,684.0,591.0,668.0
기업수,1594.0,1605.0,1620.0,1659.0,1731.0,1787.0,1865.0,1919.0,1977.0,2026.0,2056.0


In [16]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [17]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [18]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [19]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8492776190104702
1.0


0.10053906591008294

In [20]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

당기순이익/총자본                  0.198601
영업이익/총자본                   0.187893
총자본                        0.112194
판매 및 관리비/총자본               0.098901
영업현금흐름/총자본                 0.085745
최대주주비율                     0.084928
매출총이익/총자본                  0.068639
차입금의존도                     0.055151
매출채권/총자본                   0.030151
CB금액/총자본                   0.025671
결산기간_주가변동률                 0.014834
최대주주비율_변동성                 0.012021
조회공시요구카운트                  0.006668
최대주주변경카운트                  0.004701
BW금액/총자본                   0.002535
유상증자금액/총자본                 0.002524
불성실공시법인지정카운트               0.002472
결산기간_거래량증가                 0.002406
타법인주식및출자증권취득결정금액/총자본       0.002179
최대주주변경을수반하는주식양수도금액/총자본     0.001025
최대주주변경을수반하는주식담보제공금액/총자본    0.000761
dtype: float64

예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [21]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2013-01예측,2014-01예측,2015-01예측,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.857143,0.930233,0.047619,0.813953,0.076923,0.813953,0.067416,0.091954,0.738462
1,0.057143,0.046512,0.023810,0.093023,0.096154,0.116279,0.898876,0.816092,0.123077
2,0.085714,0.023256,0.928571,0.093023,0.826923,0.069767,0.033708,0.091954,0.138462


고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [22]:
dict = evaluate.see_cluster_info(model_save_dict, eval_one, drop_togehter)

data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)
data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))
data.T

,BW금액/총자본,CB금액/총자본,유상증자금액/총자본,최대주주변경을수반하는주식담보제공금액/총자본,최대주주변경을수반하는주식양수도금액/총자본,타법인주식및출자증권취득결정금액/총자본,당기순이익/총자본,매출채권/총자본,매출총이익/총자본,영업이익/총자본,...,판매 및 관리비/총자본,불성실공시법인지정카운트,조회공시요구카운트,최대주주변경카운트,결산기간_거래량증가,결산기간_주가변동률,차입금의존도,총자본,최대주주비율,최대주주비율_변동성
2012-01,81.810668,24.371573,160.444797,0.000000,35.764067,58.295079,-0.113842,0.601593,0.055353,-0.082205,...,0.137558,0.143192,0.530516,0.161972,1.122214,0.933500,31.270827,6.837327e+07,30.190370,-0.00113
2013-01,102.051096,30.048016,53.859441,0.000000,27.955764,87.784350,-0.308796,0.949037,0.082859,-0.102710,...,0.185576,0.103306,0.464876,0.167355,inf,2.101050,34.152352,1.651435e+08,30.504840,0.00569
2014-01,20.141318,50.256247,125.512211,0.000000,399.108499,68.883686,-1.534104,1.153314,-0.544902,-1.411499,...,0.866597,0.091102,0.415254,0.165254,inf,1.964250,35.717353,1.930215e+08,30.336324,inf
2015-01,4.303577,133.377321,239.206544,6.060102,75.114873,66.064755,-0.579667,1.183359,0.009784,-0.808094,...,0.817878,0.106904,0.536748,0.262806,inf,1.371350,32.077745,1.229058e+08,29.404986,inf
2016-01,13.493095,255.483640,56.110080,49.042562,64.688113,55.752267,-0.062994,0.418106,0.067277,-0.029043,...,0.096339,0.134855,0.541494,0.327801,inf,0.962995,29.520814,6.236416e+07,27.615666,inf
2017-01,12.103775,196.015886,21.854566,51.514063,73.990457,50.906580,-0.142307,0.365613,0.068205,-0.046382,...,0.114588,0.132762,0.387580,0.291221,inf,0.752128,26.947168,5.387625e+07,25.869596,inf
2018-01,19.700973,308.897146,35.240504,45.869039,94.118343,83.141835,-0.077582,0.411970,0.062241,-0.038272,...,0.100512,0.154739,0.251451,0.297872,inf,0.972769,28.875783,5.580766e+07,26.623650,inf
2019-01,24.187892,294.654977,55.970497,53.508657,63.989900,83.485001,-0.113500,0.348452,0.066821,-0.055768,...,0.122588,0.178431,0.184314,0.247059,inf,0.646268,30.456451,6.212119e+07,27.228881,inf
2020-01,13.874266,233.988706,84.177077,52.383603,37.955802,60.853366,-0.134516,0.315869,0.062941,-0.058007,...,0.120770,0.181818,0.278293,0.235622,inf,0.431695,30.497843,6.788172e+07,25.928948,inf
2021-01,30.049117,367.598908,123.448002,56.112765,51.831940,59.764796,-0.107272,0.326438,0.062237,-0.050856,...,0.112271,0.137056,0.174281,0.216582,inf,0.774223,28.892631,7.034774e+07,27.748484,inf


In [23]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [24]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,-0.337708,0.016541,0.098581,1,2.5
A000040,0.238286,0.480554,0.414295,2,2.5
A000050,-0.066685,-0.555314,0.349358,0,2.5
A000070,-0.605534,-0.135891,0.336722,1,2.5
A000080,-0.700384,0.484397,0.309707,1,2.5
...,...,...,...,...,...
A353810,-0.713690,0.736112,-0.289318,1,2.5
A354200,0.880895,-0.223943,-0.192978,2,2.5
A357780,-0.865689,-0.245984,-0.295204,1,2.5
A363280,-0.100333,-0.894652,0.143431,0,2.5
